<a href="https://colab.research.google.com/github/littlejacinthe/Style_Transfer/blob/main/STN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip uninstall tensorflow -y

Uninstalling tensorflow-2.5.0:
  Successfully uninstalled tensorflow-2.5.0


In [2]:
! pip install tensorflow-gpu==1.14.0

     |████████████████████████████████| 377.1MB 40kB/s 
     |████████████████████████████████| 491kB 34.4MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 3.2MB 33.7MB/s 
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0


In [3]:
import tensorflow as tf

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [4]:
import librosa
from IPython.display import Audio, display
import numpy as np
from PIL import Image

In [5]:
# Reads wav file and produces spectrum
N_FFT = 2048
def read_audio_spectum(filename):
    x, fs = librosa.load(filename) 
    max_len = fs * 10
    sig = x[:max_len]
    S = librosa.stft(sig, N_FFT) # short fourier transform produces spectrogram
    p = np.angle(S) # The counterclockwise angle from the positive real axis on the complex plane in the range (-pi, pi], with dtype as numpy.float64.
    S = np.log1p(np.abs(S[:,:430]))
    # Log1p : Return the natural logarithm of one plus the input array, element-wise.
    # Abs : Calculate the absolute value element-wise.
    return S

In [6]:
audio_path = '/content/classical_03.wav' 

spec = read_audio_spectum(audio_path) # get spectrogram

print("Input Shape: {}".format(spec.shape))

Input Shape: (1025, 430)


In [7]:
input = np.ascontiguousarray(spec.T[None,:,:]) # add one dimension for batch

In [8]:
N_FILTERS = 4096
N_SAMPLES = 430 # timestamp
N_CHANNELS = 1025 # frequency 

a_content_tf = np.ascontiguousarray(spec.T[None,None,:,:])

In [11]:
with tf.Graph().as_default():
  x = tf.placeholder('float32', [1,N_SAMPLES,N_CHANNELS], name="x") # using content as input for our localization network
  
  # localization network : flatten and use a one dense layer
  flat = tf.compat.v1.layers.flatten(inputs=x) # flatten to prepare the input for a dense layer
  dense = tf.compat.v1.layers.dense(inputs=flat, units=3, bias_initializer=tf.constant_initializer(value=[1, 0, 0])) 
  # use the bias initializer for the transformation. Here identity transform
  # Translation : [[1., 0., 0.5], [0., 1., 0.]]
  #Rotation : [[0.707, -0.707, 0.], [0.707, 0.707, 0.]]
  
  with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        theta = dense.eval(feed_dict={x: input}) 

In [12]:
M = np.resize(theta, (1, 2))

In [21]:
# create normalized 1D grid
x = np.linspace(-1, 1, N_SAMPLES)

In [146]:
# reshape to (2, x)
ones = np.ones(np.prod(x.shape))
print(ones.shape)
sampling_grid = np.vstack([x, ones])
print(sampling_grid.shape)

(430,)
(2, 430)


In [23]:
num_batch=1

In [24]:
# repeat grid num_batch times
sampling_grid = np.resize(sampling_grid, (num_batch, 2, 1*430))
print(sampling_grid.shape)

(1, 2, 430)


In [110]:
# transform the sampling grid i.e. batch multiply
batch_grids = np.matmul(M, sampling_grid)
print(batch_grids.shape)

# reshape to (num_batch, 1, N_SAMPLES)
batch_grids = batch_grids.reshape(1, 1, N_SAMPLES)
batch_grids = np.moveaxis(batch_grids, 1, -1)

(1, 1, 430)


In [111]:
batch_grids = batch_grids.reshape([1, 430])

In [112]:
x_s = batch_grids
# rescale x to [0, N_SAMPLES]
x = ((x_s + 1.) * N_SAMPLES) * 0.5

In [113]:
x.shape

(1, 430)

In [114]:
# grab 2 nearest corner points for x_i
x0 = np.floor(x).astype(np.int64)
x1 = x0 + 1

In [115]:
# make sure it's inside range [0, N_SAMPLES] 
x0 = np.clip(x0, 0, N_SAMPLES-1)
x1 = np.clip(x1, 0, N_SAMPLES-1)

In [130]:
# look up pixel values at corner coords
Ia = input[np.arange(num_batch)[:, None], x0]
Ib = input[np.arange(num_batch)[:, None], x1]

In [137]:
Ia.shape

(1, 430, 1025)

In [147]:
# calculate deltas (weight coefficients)
wa = (x1-x) 
wb = (x-x0)

In [148]:
wa.shape

(1, 430)

In [149]:
# add dimension for addition
wa = np.expand_dims(wa, axis=2)
wb = np.expand_dims(wb, axis=2)

In [150]:
# compute output
out = wa*Ia + wb*Ib

In [151]:
out

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])